In [10]:
import polars as pl
from benchmark_utils import mock_snakemake

if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("process_matpower_case")

In [11]:
with open(snakemake.input[0]) as f:
    matpower_case = f.read()
matpower_case[:100]

"%% MATPOWER Case Format : Version 2\nfunction mpc = cal_grid_dcopf_v2\nmpc.version = '2';\n\n%%-----  Po"

In [12]:
tables = {}
ignored_sections = ("version", "baseMVA")

header = None

for i, section in enumerate(matpower_case.split("\nmpc.")):
    if i == 0:
        continue  # before first element nothing to do
    section_name, _, section_content = section.partition("=")
    section_content, _, next_header = section_content.partition(";")

    section_name = section_name.strip()
    section_content = section_content.strip("[]\n ")
    next_header = next_header.strip()

    print(f"Processing: {section_name}")

    if section_name in ignored_sections:
        header = next_header
        continue

    assert header is not None, f"Last section should not be: {section_name}"

    header = header.split("\n")
    header = [h.strip() for h in header]
    header = [h for h in header if h.startswith("%")]
    header = header[-1].strip("%").strip()
    header = header.split()

    section_content = [
        [val for val in row.strip().split("\t")] for row in section_content.split("\n")
    ]

    table = pl.DataFrame(section_content, schema=header, orient="row")

    tables[section_name] = table

    header = next_header

tables

Processing: version
Processing: baseMVA
Processing: bus
Processing: gen
Processing: branch
Processing: gencost


{'bus': shape: (8_870, 13)
 ┌───────┬──────┬────────────┬────────────┬───┬────────┬──────┬──────┬──────┐
 │ bus_i ┆ type ┆ Pd         ┆ Qd         ┆ … ┆ baseKV ┆ zone ┆ Vmax ┆ Vmin │
 │ ---   ┆ ---  ┆ ---        ┆ ---        ┆   ┆ ---    ┆ ---  ┆ ---  ┆ ---  │
 │ str   ┆ str  ┆ str        ┆ str        ┆   ┆ str    ┆ str  ┆ str  ┆ str  │
 ╞═══════╪══════╪════════════╪════════════╪═══╪════════╪══════╪══════╪══════╡
 │ 1     ┆ 1    ┆ 56.6305477 ┆ 16.8298619 ┆ … ┆ 115.0  ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ 2     ┆ 1    ┆ 12.1925924 ┆ 2.53165799 ┆ … ┆ 66.0   ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ 3     ┆ 1    ┆ 0.0        ┆ 0.0        ┆ … ┆ 66.0   ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ 4     ┆ 1    ┆ 22.7970837 ┆ 8.37324562 ┆ … ┆ 230.0  ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ 5     ┆ 1    ┆ 115.972748 ┆ 31.4555386 ┆ … ┆ 115.0  ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ …     ┆ …    ┆ …          ┆ …          ┆ … ┆ …      ┆ …    ┆ …    ┆ …    │
 │ 8866  ┆ 1    ┆ 0.0        ┆ 0.0        ┆ … ┆ 66.0   ┆ 0    ┆ 1.06 ┆ 0.94 │
 │ 8867  ┆ 1    ┆ 0.0        ┆ 0.0   

In [13]:
tables2 = {}

for table_name in tables:
    table = tables[table_name]
    for col in table.columns:
        table = table.with_columns(
            pl.col(col).cast(pl.Float64)
        )  # we do not expect any strings!
        if (table[col].round() == table[col]).all():
            table = table.with_columns(pl.col(col).cast(pl.Int64))
        if (table[col].is_in([0, 1])).all():
            table = table.with_columns(pl.col(col).cast(pl.Boolean))
        table = table.with_columns(pl.col(col).shrink_dtype())

    tables2[table_name] = table

tables2

{'bus': shape: (8_870, 13)
 ┌───────┬──────┬────────────┬───────────┬───┬────────┬───────┬──────┬──────┐
 │ bus_i ┆ type ┆ Pd         ┆ Qd        ┆ … ┆ baseKV ┆ zone  ┆ Vmax ┆ Vmin │
 │ ---   ┆ ---  ┆ ---        ┆ ---       ┆   ┆ ---    ┆ ---   ┆ ---  ┆ ---  │
 │ i16   ┆ i8   ┆ f32        ┆ f32       ┆   ┆ i16    ┆ bool  ┆ f32  ┆ f32  │
 ╞═══════╪══════╪════════════╪═══════════╪═══╪════════╪═══════╪══════╪══════╡
 │ 1     ┆ 1    ┆ 56.630547  ┆ 16.829863 ┆ … ┆ 115    ┆ false ┆ 1.06 ┆ 0.94 │
 │ 2     ┆ 1    ┆ 12.192593  ┆ 2.531658  ┆ … ┆ 66     ┆ false ┆ 1.06 ┆ 0.94 │
 │ 3     ┆ 1    ┆ 0.0        ┆ 0.0       ┆ … ┆ 66     ┆ false ┆ 1.06 ┆ 0.94 │
 │ 4     ┆ 1    ┆ 22.797083  ┆ 8.373245  ┆ … ┆ 230    ┆ false ┆ 1.06 ┆ 0.94 │
 │ 5     ┆ 1    ┆ 115.972748 ┆ 31.455538 ┆ … ┆ 115    ┆ false ┆ 1.06 ┆ 0.94 │
 │ …     ┆ …    ┆ …          ┆ …         ┆ … ┆ …      ┆ …     ┆ …    ┆ …    │
 │ 8866  ┆ 1    ┆ 0.0        ┆ 0.0       ┆ … ┆ 66     ┆ false ┆ 1.06 ┆ 0.94 │
 │ 8867  ┆ 1    ┆ 0.0        ┆ 0.0   

In [14]:
assert {"bus", "gen", "branch", "gencost"} == set(tables2.keys())
assert tables2["gen"].height == tables2["gencost"].height
bus, gen, branch, gencost = (
    tables2["bus"],
    tables2["gen"],
    tables2["branch"],
    tables2["gencost"],
)
assert (gencost["n"] == 3).all()
assert (gencost["2"] == 2).all()
gencost = gencost.drop("2", "n").rename(
    {"c(n-1)": "cost_a", "...": "cost_b", "c0": "cost_c"}
)
gencost

startup,shutdown,cost_a,cost_b,cost_c
bool,bool,f32,f32,f32
false,false,0.059172,14.878815,46.499573
false,false,0.082992,44.91518,1427.413086
false,false,0.082992,44.91518,1427.413086
false,false,0.078724,14.878815,34.646786
false,false,0.078724,14.878815,34.646786
…,…,…,…,…
false,false,0.0,0.0,0.0
false,false,0.0,0.0,0.0
false,false,0.0,0.0,0.0


In [15]:
gen_merged = pl.concat([gen.with_row_index(name="gen_id"), gencost], how="horizontal")
gen_merged

gen_id,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,Pc1,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf,startup,shutdown,cost_a,cost_b,cost_c
u32,i16,f32,bool,f32,f32,bool,i8,bool,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,f32,f32,f32
0,745,4.033363,false,18.777143,-14.670714,true,0,true,12.1,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.059172,14.878815,46.499573
1,1804,0.0,false,34.5,-23.700001,true,0,true,71.199997,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.082992,44.91518,1427.413086
2,1804,0.0,false,34.5,-23.700001,true,0,true,71.199997,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.082992,44.91518,1427.413086
3,1964,0.366669,false,18.777143,-14.670714,true,0,true,1.1,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.078724,14.878815,34.646786
4,1964,0.366669,false,18.777143,-14.670714,true,0,true,1.1,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.078724,14.878815,34.646786
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3887,8862,0.0,false,200.0,-200.0,true,100,true,0.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.0,0.0,0.0
3888,8863,0.0,false,200.0,-200.0,true,100,true,0.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.0,0.0,0.0
3889,8864,0.0,false,200.0,-200.0,true,100,true,0.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,0.0,0.0,0.0


In [16]:
tables3 = (bus, branch, gen_merged)
tables4 = []
for table in tables3:
    to_drop = []
    for col in table.columns:
        if len(table[col].unique()) == 1:
            to_drop.append(col)
    tables4.append(table.drop(*to_drop))

bus, branch, gen_merged = tables4
tables4

[shape: (8_870, 6)
 ┌───────┬──────┬────────────┬───────────┬───────────┬────────┐
 │ bus_i ┆ type ┆ Pd         ┆ Qd        ┆ Va        ┆ baseKV │
 │ ---   ┆ ---  ┆ ---        ┆ ---       ┆ ---       ┆ ---    │
 │ i16   ┆ i8   ┆ f32        ┆ f32       ┆ f32       ┆ i16    │
 ╞═══════╪══════╪════════════╪═══════════╪═══════════╪════════╡
 │ 1     ┆ 1    ┆ 56.630547  ┆ 16.829863 ┆ -1.939806 ┆ 115    │
 │ 2     ┆ 1    ┆ 12.192593  ┆ 2.531658  ┆ -1.733525 ┆ 66     │
 │ 3     ┆ 1    ┆ 0.0        ┆ 0.0       ┆ -1.763379 ┆ 66     │
 │ 4     ┆ 1    ┆ 22.797083  ┆ 8.373245  ┆ -1.802757 ┆ 230    │
 │ 5     ┆ 1    ┆ 115.972748 ┆ 31.455538 ┆ -1.897178 ┆ 115    │
 │ …     ┆ …    ┆ …          ┆ …         ┆ …         ┆ …      │
 │ 8866  ┆ 1    ┆ 0.0        ┆ 0.0       ┆ -0.169835 ┆ 66     │
 │ 8867  ┆ 1    ┆ 0.0        ┆ 0.0       ┆ -2.065685 ┆ 66     │
 │ 8868  ┆ 1    ┆ 0.0        ┆ 0.0       ┆ -1.981266 ┆ 115    │
 │ 8869  ┆ 1    ┆ 0.0        ┆ 0.0       ┆ -1.712662 ┆ 66     │
 │ 8870  ┆ 1    ┆ 0.0

In [17]:
bus.write_parquet(snakemake.output.bus)
branch.write_parquet(snakemake.output.branch)
gen_merged.write_parquet(snakemake.output.gen)